In [60]:
import pandas as pd
import talib
import numpy as np
import vectorbt as vbt 
from vectorbt.portfolio.enums import OrderSide
from numba import njit

# 計算keltner channel
def keltner_channel(high, low, close, timeperiod = 20, nbdev = 2):
    middle_line = talib.EMA(close, timeperiod = timeperiod)
    atr = talib.ATR(high, low, close, timeperiod = timeperiod)
    
    # 上通道和下通道
    upper_channel = middle_line + nbdev * atr
    lower_channel = middle_line - nbdev * atr
    
    return upper_channel, middle_line, lower_channel

# 讀取data
data = pd.read_csv("2363.csv")
data = data.set_index('Date')
data.index = pd.to_datetime(data.index)

# 計算指標
upper_channel, middle_line, lower_channel = keltner_channel(data['High'], data['Low'], data['Close'])
upper, middle, lower = talib.BBANDS(data['Close'], timeperiod=20, nbdevup=1.5, nbdevdn=1.5, matype=0)
k, d = talib.STOCH(data['High'],data['Low'], data['Close'], fastk_period=14, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
j = 3 * k - 2 * d
five_day_min_low = talib.MIN(data['Low'], timeperiod=5) # 信號k棒五天最低價當作long strategy的停損點
five_day_max_high = talib.MAX(data['High'], timeperiod=5) # 信號k棒五天最高價當作short strategy的停損點

# 放入dataframe
data['upper_channel'] = upper_channel
data['lower_channel'] = lower_channel
data['upper_bb'] = upper
data['lower_bb'] = lower
data['KDJ'] = j
data['five_day_min_low'] = five_day_min_low
data['five_day_max_high'] = five_day_max_high

# 在squeeze中出現close突破upper bb的上漲k棒且KDJ大於80(short)
# 在squeeze中出現close跌破lower bb的下跌K棒且KDJ小於20(long)
data['squeeze'] = (data['upper_bb'] <= data['upper_channel']) & (data['lower_bb']  >= data['lower_channel'])
data['breakthrough'] = (data['Close'] > data['Open']) & (data['Close'] >= data['upper_bb'])
data['breakdown'] = (data['Close'] < data['Open']) & (data['Close'] <= data['lower_bb'])
data['target_squeeze_long'] = data['squeeze'] & data['breakdown'] & (data['KDJ'] <= 20)
data['target_squeeze_short'] = data['squeeze'] & data['breakthrough'] & (data['KDJ'] >= 80)



long and short strategy(逆勢交易)

In [61]:
# 台股手續費
def custom_fees(order):
    if order.side == OrderSide.Buy:
        return order.size * order.price * 0.001425  
    elif order.side == OrderSide.Sell:
        return order.size * order.price * 0.001425 + order.size * order.price * 0.003  
    return 0

# long strategy
signals_long = np.zeros(len(data))
condition1_long = False
partition_size_long = 0
stop_loss_long = 0
for i in range(len(data)):
    if data['target_squeeze_long'][i] and not condition1_long:
        condition1_long = True
        stop_loss_long = data['five_day_min_low'][i]
    if partition_size_long == 0:
        crossover = (data['KDJ'][i-1] <= 20) & (data['KDJ'][i] > 20)
        if condition1_long and crossover:
            signals_long[i] = 1
            partition_size_long = 1
    if partition_size_long == 1:
        crossunder = (data['KDJ'][i-1] >= 80) & (data['KDJ'][i] < 80)
        if crossunder or (data['Close'][i] <= stop_loss_long):# 停利停損
            signals_long[i] = -1
            condition1_long = False # condition1重置
            stop_loss_long = 0 # 停損點重置
            partition_size_long = 0 # 平倉

# short strategy
signals_short = np.zeros(len(data))
condition1_short = False
partition_size_short = 0
stop_loss_short = 0
for i in range(len(data)):
    if data['target_squeeze_short'][i] and not condition1_short:
        condition1_short = True
        stop_loss_short = data['five_day_max_high'][i]
    if partition_size_short == 0:
        crossunder = (data['KDJ'][i-1] >= 80) & (data['KDJ'][i] < 80)
        if condition1_short and crossunder:
            signals_short[i] = -1
            partition_size_short = -1
    if partition_size_short == -1:
        crossover = (data['KDJ'][i-1] <= 20) & (data['KDJ'][i] >20 )
        if crossover or (data['Close'][i] >= stop_loss_short):# 停利停損
            signals_short[i] = 1
            condition1_short = False # condition1重置
            stop_loss_short = 0 # 停損點重置
            partition_size_short = 0 # 平倉

entries_long = signals_long == 1
exits_long = signals_long == -1
entries_short = signals_short == -1
exits_short = signals_short == 1
pf_long = vbt.Portfolio.from_signals(data['Close'], 
                                     entries_long, 
                                     exits_long, 
                                     entries_short, 
                                     exits_short, 
                                     fees = 0.001425, 
                                     freq='1D' , 
                                     direction='both')
print(pf_long.stats().to_string()) # to_string()可以將全部結果攤開
'''entries_short = signals_short == -1
exits_short = signals_short == 1
pf_short = vbt.Portfolio.from_signals(data['Close'], entries_short, exits_short, direction='shortonly')'''
# print(pf_short.stats().to_string())


C:\Users\Admin\AppData\Local\Temp\ipykernel_32492\2117740496.py:15: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Admin\AppData\Local\Temp\ipykernel_32492\2117740496.py:19: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Admin\AppData\Local\Temp\ipykernel_32492\2117740496.py:17: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Admin\AppData\Local\Temp\ipykernel_32492\2117740496.py:24: FutureWarning:

Series.__g

Start                               2016-01-04 00:00:00
End                                 2023-11-21 00:00:00
Period                               1915 days 00:00:00
Start Value                                       100.0
End Value                                    201.638302
Total Return [%]                             101.638302
Benchmark Return [%]                          639.88006
Max Gross Exposure [%]                            100.0
Total Fees Paid                               26.712156
Max Drawdown [%]                              36.632709
Max Drawdown Duration                 759 days 00:00:00
Total Trades                                         49
Total Closed Trades                                  49
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  53.061224
Best Trade [%]                                24.259534
Worst Trade [%]                               -9

C:\Users\Admin\AppData\Local\Temp\ipykernel_32492\2117740496.py:57: UserWarning:

direction has no effect if short_entries and short_exits are set



"entries_short = signals_short == -1\nexits_short = signals_short == 1\npf_short = vbt.Portfolio.from_signals(data['Close'], entries_short, exits_short, direction='shortonly')"

visualization

In [62]:
pf_long.plot().show()
# pf_short.plot().show()